# 多层感知机模型示例

## Neural Network Overview

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>

## MNIST Dataset Overview

MNIST图像数据集使用形如［28，28］的二阶数组来表示每张图像，数组中的每个元素对应一个像素点。该数据集中的图像都是256阶灰度图，像素值0表示白色（背景），255表示黑色（前景）。由于每张图像的尺寸都是28x28像素，为了方便连续存储，我们可以将形如［28，28］的二阶数组“摊平”成形如［784］的一阶数组。数组中的784个元素共同组成了一个784维的向量。

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

More info: http://yann.lecun.com/exdb/mnist/

In [23]:
from __future__ import print_function

# 导入 MNIST 数据集
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf
tf.__version__

# tf2 代码
mint = tf.keras.datasets.mnist
(x_,y_),(x_1,y_1) = mint.load_data()
import matplotlib.pyplot as plt
plt.imshow(x_[0], cmap="binary")

In [18]:
# 超参数
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# 神经网络参数
n_hidden_1 = 256 # 第一层神经元个数
n_hidden_2 = 256 # 第二层神经元个数
num_input = 784 # MNIST 输入数据(图像大小: 28*28)
num_classes = 10 # MNIST 手写体数字类别 (0-9)

# 输入到数据流图中的训练数据
# X = tf.placeholder("float", [None, num_input])
# Y = tf.placeholder("float", [None, num_classes])
# tf2 代码
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder("float", [None, num_input])
Y = tf.compat.v1.placeholder("float", [None, num_classes])

In [19]:
# 权重和偏置
weights = {
    'h1': tf.Variable(tf.compat.v1.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.compat.v1.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.compat.v1.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.compat.v1.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.compat.v1.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.compat.v1.random_normal([num_classes]))
}

In [20]:
# 定义神经网络
def neural_net(x):
    # 第一层隐藏层（256个神经元）
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # 第二层隐藏层（256个神经元）
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # 输出层
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [21]:
# 构建模型
logits = neural_net(X)

# 定义损失函数和优化器
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 定义预测准确率
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 初始化所有变量（赋默认值）
init = tf.compat.v1.global_variables_initializer()

In [25]:
# 开始训练
with tf.compat.v1.Session() as sess:

    # 执行初始化操作
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # 执行训练操作，包括前向和后向传播
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # 计算损失值和准确率
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # 计算测试数据的准确率
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

AttributeError: module 'keras.api._v2.keras.datasets.mnist' has no attribute 'train'